In [1]:
from multiprocessing import Pool
import numpy as np
from functools import reduce
from bs4 import BeautifulSoup
from selenium import webdriver
import string
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import requests
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from difflib import SequenceMatcher
import requests
from unidecode import unidecode

In [2]:
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)

#defining preprocessing of string fuction

def preprocessing(text):
  # Replace of unicode characters
  text = unidecode(text,errors='replace')

  # Convert all text to lowercase
  text = text.lower()
  
  # Tokenize the text
  tokens = word_tokenize(text)

  # Remove punctuation
  tokens = [token.translate(table) for token in tokens]

  # Remove blank space tokens
  tokens = [token for token in tokens if token.strip() and len(token) > 1]

  # returning space saperated string of tokens
  return ' '.join(tokens)


In [3]:

# sample input 1
product_links = [
    "https://www.woolsboutiqueuomo.com/products/grenadine-silk-tie-blue-wools-boutique-uomo",
    "https://www.woolsboutiqueuomo.com/products/canary-yellow-grenadine-silk-tie-gauze",
    "https://www.woolsboutiqueuomo.com/products/grenadine-silk-tie-chunky-gauze-copper-brown",
    "https://www.woolsboutiqueuomo.com/products/brick-red-grenadine-silk-tie-chunky-gauze",
    "https://www.woolsboutiqueuomo.com/products/fmarino-checked-wool-tie-hollandsherry-blue",
    "https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-bronze",
    "https://www.woolsboutiqueuomo.com/products/glen-check-tie-wool-curry-fmarino-napoli",
    "https://www.woolsboutiqueuomo.com/products/regimental-gauze-silk-tie-denim-blue",
    "https://www.woolsboutiqueuomo.com/products/burgundy-grenadine-silk-tie-chunky-gauze",
    "https://www.woolsboutiqueuomo.com/products/cobalt-blue-silk-tie-chunky-gauze-fmarino",
    "https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-dark-blue",
    "https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-grass-green",
    "https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-mud-brown",
    "https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-pine-green"
]

# sample input 2
product_links = [
    "https://www.woolsboutiqueuomo.com/products/brown-wool-crewneck-sweater-cruciani",
    "https://www.woolsboutiqueuomo.com/products/pervinca-blue-cotton-sweater-cruciani",
    "https://www.woolsboutiqueuomo.com/products/army-green-cotton-crewneck-sweater",
    "https://www.woolsboutiqueuomo.com/products/green-cotton-polo-shirt-cruciani",
    "https://www.woolsboutiqueuomo.com/products/steel-blue-merino-wool-crewneck-sweater",
    "https://www.woolsboutiqueuomo.com/products/mens-white-polo-long-sleeve-cruciani",
    "https://www.woolsboutiqueuomo.com/products/cruciani-mens-wool-vests-blu",
    "https://www.woolsboutiqueuomo.com/products/cruciani-mens-wool-sweater-crewneck-raglan-sleeve-blu",
    "https://www.woolsboutiqueuomo.com/products/cruciani-mens-vneck-sweater-wool-blu",
    "https://www.woolsboutiqueuomo.com/products/turtleneck-wool-jumper-dark-blue-cruciani",
    "https://www.woolsboutiqueuomo.com/products/turtleneck-wool-jumper-smoke-grey-cruciani",
    "https://www.woolsboutiqueuomo.com/products/army-green-crewneck-jumper-cashmere-silk-cruciani",
    "https://www.woolsboutiqueuomo.com/products/cruciani-sweater-cashmere-silk-grey",
    "https://www.woolsboutiqueuomo.com/products/cruciani-wool-cardigan-blue",
    "https://www.woolsboutiqueuomo.com/products/cruciani-mens-cashmere-sweater-blue",
    "https://www.woolsboutiqueuomo.com/products/cruciani-crewneck-cashmere-silk-marino",
    "https://www.woolsboutiqueuomo.com/products/cruciani-sweater-cashmere-silk-nightblue",
    "https://www.woolsboutiqueuomo.com/products/cruciani-vneck-sweater-cashmere-silk-grey",
    "https://www.woolsboutiqueuomo.com/products/cruciani-sweater-cashmere-silk-marino-blue",
    "https://www.woolsboutiqueuomo.com/products/cruciani-vneck-sweater-cashmere-silk-blue"
]

#sample input 3
product_links = [
    "https://www.woolsboutiqueuomo.com/products/tshirt-ossigeno-cotton-army-green-cruciani",
    "https://www.woolsboutiqueuomo.com/products/cruciani-linen-sailor-tshirt-marinara",
    "https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-long-sleeve-blu",
    "https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-long-sleeve-white",
    "https://www.woolsboutiqueuomo.com/products/cruciani-tshirt-ossigeno-cotton-white",
    "https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-blue",
]

In [4]:
def preProcessList(product_list):
    global base_url
    base_url = '/'.join(product_list[0].split('/')[:3])+'/'
    products = list(map(lambda x:x.split('/')[-1],product_list))
    return products


In [5]:
fetch_data_org = []
handleToIdConvertor = {}

def fetch(url):
    global fetch_data_org, handleToIdConvertor, product_handle_list
    response = requests.get(url + 'collections/all/products.json?page=1').json()
    i=2
    while response['products']:
        fetch_data_org += response['products']
        response = requests.get(url + 'collections/all/products.json?page=' + str(i)).json()
        i += 1

    for i in fetch_data_org:
        if i['handle'] in product_handle_list:
            handleToIdConvertor[i['handle']] = i


In [6]:
def preProcessProductDetails(product):
    preProcessedVendor = preprocessing(product['vendor'])
    preProcessedProductType = TextBlob(preprocessing(product['product_type'])).words
    preProcessedProductTypeSingular = preProcessedProductType.pluralize()[0]
    preProcessedProductTypePlural = preProcessedProductType.singularize()[0]
    preProcessedHandle = preprocessing(product['handle'])
    preProcessedTitle = preprocessing(product['title']).split(' ') # list

    #remove brand from title if present
    for word in preProcessedTitle: 
        if (preProcessedVendor in word or word in preProcessedVendor) or (''.join(preProcessedVendor.split(' ')) in word or word in ''.join(preProcessedVendor.split(' '))):
            preProcessedTitle.remove(word)
    
    # remove product type from title if present
    for word in preProcessedTitle:
        if (preProcessedProductTypeSingular in word or preProcessedProductTypePlural in word) or (word in preProcessedProductTypeSingular or word in preProcessedProductTypePlural):
            preProcessedTitle.remove(word)

    return ' '.join(preProcessedTitle)
    


In [7]:
base_url = ''
product_handle_list = preProcessList(product_links)
fetch(base_url)


In [8]:
for product in handleToIdConvertor.values():
    product['preprocessed_title'] = preProcessProductDetails(product)


In [9]:
handleToIdConvertor

{'cruciani-linen-sailor-tshirt-marinara': {'id': 8138810360102,
  'title': 'Cruciani Linen T-Shirt Pocket Sailor Navy Blue Striped',
  'handle': 'cruciani-linen-sailor-tshirt-marinara',
  'body_html': '<meta charset="utf-8">\n<p>Men\'s crewneck t-shirt made by Cruciani with soft and light linen, smooth thin and very breathable, a feeling of freshness and softness on the skin, comfortable fit ideal to wear in your free time.</p>\n<p><strong>FIT &amp; DETAILS</strong><br></p>\n<meta charset="utf-8">\n<ul>\n<li>\n<meta charset="utf-8">Made in Italy</li>\n<li>\n<span>95%\xa0linen\xa0 5%\xa0elastan</span><br>\n</li>\n<li>Sailor <em>marinara</em> navy blue and white color striped</li>\n<li>Pocket in the front</li>\n<li>Regular fit, short sleeve</li>\n<li>Ideal to wear on the skin</li>\n<li>Ideal to wear in the\xa04 seasons</li>\n</ul>\n<p><strong>CARE</strong></p>\n<ul>\n<li>Maximum washing temperature 30°C</li>\n<li>Do not bleach</li>\n<li>Do not tumble dry</li>\n<li>Do not dry with strong 

In [10]:
def FindKeyDifference():
    final_product_list = list(map(lambda x: {'key_difference':x['preprocessed_title'].split(),'product_link':base_url+'products/'+x['handle'],'product_title':x['title']}, handleToIdConvertor.values()))
    all_keywords = set(list(reduce(lambda a, b: a+b, list(map(lambda x:x['key_difference'],final_product_list)))))

    for keyword in all_keywords:

        title_containing_keyword = []
        for product_index, product_details in enumerate(final_product_list):
            if keyword in product_details['key_difference']:
                title_containing_keyword.append(product_index)

        if len(title_containing_keyword) > len(final_product_list)*0.5:
            for index in title_containing_keyword:
                final_product_list[index]['key_difference'].remove(keyword)

    for product in final_product_list:
        product['key_difference'] = ' '.join(list(map(lambda x:x[0].upper()+x[1:],product['key_difference'])))

    return final_product_list


In [11]:
FindKeyDifference()

[{'key_difference': 'Linen Pocket Sailor Navy Blue Striped',
  'product_link': 'https://www.woolsboutiqueuomo.com/products/cruciani-linen-sailor-tshirt-marinara',
  'product_title': 'Cruciani Linen T-Shirt Pocket Sailor Navy Blue Striped'},
 {'key_difference': 'Army Green',
  'product_link': 'https://www.woolsboutiqueuomo.com/products/tshirt-ossigeno-cotton-army-green-cruciani',
  'product_title': "Cruciani Men's T-Shirt Ossigeno Cotton Army Green"},
 {'key_difference': 'Blue',
  'product_link': 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-blue',
  'product_title': "Cruciani Men's T-Shirt Ossigeno Cotton Blue"},
 {'key_difference': 'Long Sleeve Blue',
  'product_link': 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-long-sleeve-blu',
  'product_title': "Cruciani Men's T-Shirt Ossigeno Cotton Long Sleeve Blue"},
 {'key_difference': 'White',
  'product_link': 'https://www.woolsboutiqueuomo.com/products/cruciani-tshirt-o